In [ ]:
from astropy import units as u
from colorcet import cm
from eigenroutines import *
from gc import collect
from matplotlib import pyplot as plt
import numpy as np
from os import mkdir, path
from pandas import DataFrame
import pickle
from parameter_functions import *
import starry
from system_properties import warm_Jupiter as system_properties
from warnings import filterwarnings

%matplotlib inline

starry.config.lazy = False
starry.config.quiet = True

filterwarnings("ignore", category=FutureWarning)

plt.rc("text", usetex=True)
plt.rc("font", family="serif")
plt.rc("axes", labelsize=19)
plt.rc("xtick", labelsize=15)
plt.rc("ytick", labelsize=15)

In [ ]:
max_degree = 20
num_eigencurves = 50
# num_eigencurves = (max_degree+1)**2 - 1

In [ ]:
time_extent = "T1-T4"
# Number of points per orbital period.
time_resolution = 12500

In [ ]:
preferred_colormap = plt.get_cmap("Purples_r")
saved_filetypes = ["pdf", "png"]
folder_name = "2021-01-10_rotrate_{}-curves_deg{}".format(num_eigencurves, max_degree)
save_directory = "Plots/WJ/" + folder_name
if not path.isdir(save_directory):
    mkdir(save_directory)

In [ ]:
system = generate_system(system_properties, max_degree)
star = system.primary
planet = system.secondaries[0]

In [ ]:
eclipse_timings = eclipse_durations(system)

In [ ]:
print(eclipse_timings)

In [ ]:
# Scales are the number of various eclipse intervals per orbital period.
rotation_scale = rotation_parameter(system, timescale="ingress")
rotation_scale_total = rotation_parameter(system, timescale="total")
print(rotation_scale)
print(rotation_scale_total)
print(rotation_scale/rotation_scale_total)

In [ ]:
# cosine_obliquities = np.linspace(1, 0, num=6)
# obliquities = np.rad2deg(np.arccos(cosine_obliquities)) * u.deg
# parameter_values = obliquities
# function_to_vary_parameter = vary_obliquity_skyplane

rotation_ratios = np.r_[np.array([1e-4, 1]), np.logspace(-7, 6, num=14, base=2)*rotation_scale]
parameter_values = rotation_ratios
function_to_vary_parameter = vary_rotation_rate_ratio

In [ ]:
times = cut_to_eclipse(system=system,
                       time_resolution=time_resolution,
                       extent=time_extent)

In [ ]:
save_name = system_properties["name"] + "." + folder_name

In [ ]:
try:
    mode_sets = pickle.load(open(save_name+".p", "rb"))
except:
    print("Didn't find the mode sets! Generating...")
    mode_sets = generate_set_of_eigenmodes(system=system,
                                           times=times,
                                           function_to_vary_parameter=function_to_vary_parameter,
                                           parameter_values=parameter_values,
                                           max_degree=max_degree,
                                           num_eigencurves=num_eigencurves)

    if save_name is not None:
        pickle.dump(mode_sets, open(save_name+".p", "wb"))

In [ ]:
component_sets = mode_sets["component sets"]
harmonic_sets = mode_sets["harmonic sets"]
score_sets = mode_sets["score sets"]
fixed_labels = mode_sets["fixed labels"]
variable_labels = mode_sets["variable labels"]

In [ ]:
plot_labels = [r"$\ell_\mathrm{{max}}={}$".format(max_degree), *fixed_labels]
plot_title = ", ".join(plot_labels)

file_label = plot_title\
    .replace("$", "")\
    .replace("=", "")\
    .replace(", ", "_")\
    .replace(" ", "")\
    .replace(r"\ell_\mathrm{max}", "deg")\
    .replace(r"\omega_\mathrm{rot}/\omega_\mathrm{orb}", "rot-orb-ratio")\
    .replace(r"\psi_\mathrm{LOS}", "obl-LOS")\
    .replace(r"\psi_\mathrm{sky}", "obl-on-sky")\
    .replace(r"^\circ", "deg")

In [ ]:
# Whether to plot the eigencurves themselves, or to plot their differences from the non-rotating case.
plot_difference = False
curve_cutoff = 25

colors = [preferred_colormap(value) for value in np.linspace(0, 0.75, num=np.min([num_eigencurves, curve_cutoff]))]
times_from_eclipse = times - 0.5*planet.porb
fig, axes = plt.subplots(4, 4, figsize=(16, 32), sharex=True)
for ax, components, harmonics, variable_label in zip(axes.flatten()[:],
                                                     component_sets,
                                                     harmonic_sets,
                                                     variable_labels):
    for i, (component, color) in enumerate(zip(components[:curve_cutoff], colors)):
        if i == 0:
            plot_offset = 0
        else:
            plot_offset = np.nanmax(plotted_curve)
        eigencurve = -np.sum(component*harmonics, axis=1)
        if plot_difference:
            nonrotating_eigencurves = np.sum(component_sets[0][i]*harmonic_sets[0], axis=1)
            differences = eigencurve + np.array([1, -1])[:, np.newaxis]*nonrotating_eigencurves
            which_difference = np.argmin(np.sum(differences**2, axis=-1))
            plotted_curve = differences[which_difference] + (plot_offset+0.1)
            ax.plot(times_from_eclipse,
                    plotted_curve,
                    color=color)
        else:
            plotted_curve = eigencurve + (plot_offset+0.1)
            ax.plot(times_from_eclipse,
                    plotted_curve,
                    color=color)
        ax.yaxis.set_visible(False)
        ax.set_title(variable_label, fontsize=11)

#axes.flatten()[-1].axis("off")
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none',
                top=False, bottom=False,
                left=False, right=False)
plt.xlabel(r"Time relative to eclipse (days)", fontsize=16)
plt.ylabel(r"$F_\mathrm{P}/F_\mathrm{\star}$ (arbitrary scale)", fontsize=16)
plt.title(plot_title, pad=30, fontsize=14)
for filetype in saved_filetypes:
    plt.savefig(save_directory + "/" + system_properties["name"] +\
                "_" + file_label + "_eigencurves.{}".format(filetype),
                bbox_inches="tight", dpi=300)

In [ ]:
for eigenmode in range(num_eigencurves):
    plt.clf()
    plt.close("all")
    collect(2)
    fig, axes = plt.subplots(4, 4, figsize=(12, 8), sharex=True, sharey=True)
    for ax, component, score, parameter_value, variable_label in zip(axes.flatten()[:],
                                                                     component_sets[:, eigenmode, ...],
                                                                     score_sets[:, eigenmode, ...], 
                                                                     parameter_values,
                                                                     variable_labels):
        function_to_vary_parameter(system, parameter_value)
        eigenmap = system.secondaries[0].map
        for l in range(1, max_degree+1):
            for m in range(-l, l+1):
                eigenmap[l, m] = component[l**2+l+m-1]
        eigenmap.show(ax=ax, cmap=preferred_colormap, theta=0, grid=False, projection="mollweide")
        ax.set_title(variable_label+r" $\left(f_\mathrm{{var}}={0:.3f}\right)$".format(score), fontsize=8)

    # axes.flatten()[-1].axis("off")
    plt.subplots_adjust(hspace=0.05, wspace=0.025)
    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none',
                    top=False, bottom=False,
                    left=False, right=False)
    plt.title("MODE {}: ".format(eigenmode) + ", ".join(fixed_labels), pad=30, fontsize=14)
    for filetype in saved_filetypes:
        plt.savefig(save_directory + "/" + system_properties["name"] +\
                    "_" + file_label + "_eigenmode{}-mollweidemaps.{}".format(eigenmode, filetype),
                    bbox_inches="tight", dpi=300)

In [ ]:
'''
plt.clf()
plt.close("all")
collect(2)
fig, axes = plt.subplots(2, 3, figsize=(9, 6), sharex=True, sharey=True)
for i, (ax, component, score, parameter_value, variable_label) in enumerate(zip(axes.flatten()[:],
                                                                            component_sets[0, :, ...],
                                                                            score_sets[0, :, ...], 
                                                                            parameter_values,
                                                                            variable_labels)):
        function_to_vary_parameter(system, parameter_values[0])
        eigenmap = system.secondaries[0].map
        for l in range(1, max_degree+1):
            for m in range(-l, l+1):
                eigenmap[l, m] = component[l**2+l+m-1]
        eigenmap.show(ax=ax, cmap=plt.get_cmap("Greys_r"), theta=180)
        ax.set_title("Mode {}".format(i))

axes.flatten()[-1].axis("off")

plt.subplots_adjust(hspace=0.20, wspace=0.05)
fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none',
                top=False, bottom=False,
                left=False, right=False)
for filetype in saved_filetypes:
    plt.savefig(save_directory + "/" + system_properties["name"] +\
                "_" + "base-globemaps.{}".format(filetype),
                bbox_inches="tight", dpi=300)
'''

In [ ]:
'''
colors = [plt.get_cmap("Greys_r")(value) for value in np.linspace(0, 0.75, num=num_eigencurves)]
times_from_eclipse = times - 0.5*planet.porb
fig, axes = plt.subplots(1, 1, figsize=(6, 6), sharex=True)
ax = axes

for i, (component, color) in enumerate(zip(component_sets[0], colors)):
    if i == 0:
        plot_offset = 0
    else:
        plot_offset = np.nanmax(plotted_curve)
    eigencurve = -np.sum(component*harmonic_sets[0], axis=1)
    plotted_curve = eigencurve-np.nanmin(eigencurve) + (plot_offset+0.1)
    ax.scatter(times_from_eclipse,
               plotted_curve,
               color=color,
               s=1)
    ax.yaxis.set_visible(False)
    # ax.set_title(variable_labels[0], fontsize=11)

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none',
                top=False, bottom=False,
                left=False, right=False)
plt.xlabel(r"Time relative to eclipse (days)", fontsize=16)
plt.ylabel(r"$F_\mathrm{P}/F_\mathrm{\star}$ (arbitrary scale)", fontsize=16)
# plt.title(plot_title, pad=30, fontsize=14)
for filetype in saved_filetypes:
    plt.savefig(save_directory + "/" + system_properties["name"] +\
                "_base_eigencurves.{}".format(filetype),
                bbox_inches="tight", dpi=300)
'''